In [18]:
import json 
import random
import numpy as np 
import copy
from joblib import parallel_backend , Parallel
from Equations_Run_Combo_V_2 import *


---
# HOD 21_01 - 22_12
---

In [ ]:
####                                       First Dist Disc   - Ran top models


with open("/Users/cs/Desktop/DATA_LSTM_ETF_V2/GS_21_01_to_22_12_chunk_1_32_10percentPOS_Vset_HOD.json" , "rb") as f:
    results = json.load(f)


scored_results = []
for entry in results:
    avg = entry['cv_sets'].get('overall_metrics')
    acc = avg.get('accuracy') or 0.0
    prec_up = avg.get('precision_up') or 0.0
    recall_up = avg.get('recall_up') or 0.0

    total_score = acc  # or add prec_up, etc.
    
    if recall_up > 1 and prec_up > 40:

        scored_results.append((total_score, entry))

# Step 2: Sort by score descending
scored_results.sort(reverse=True, key=lambda x: x[0])


top = scored_results[:90]

for i, j in top:
    print(j["combo_number"], '<--->' , j["cv_sets"]["overall_metrics"] , '<---->' , j["parameters"] )

# Step 4: Extract the parameter combos
combos_1 = [j["parameters"] for i, j in top]


len(combos_1)


# combos_test_param = []

# for i in combos:
#     i_copy = copy.deepcopy(i)
#     i_copy["val_set_fraction"] = 0.05
#     i_copy["num_folds"] = 4
#     i_copy["end_value_train_set_fraction"] = 0.95
#     combos_test_param.append(i_copy)


# #### ----------------------                 RUN COMBOS
# with parallel_backend("loky", n_jobs=3):
#     results = Parallel()(
#         delayed(distribution_discovery)(c, combo_index = i , number_of_seeds = 60)
        
#         for i, c in enumerate(combos)
#     )



297 <---> {'accuracy': 95.83333333333334, 'precision_up': 100.0, 'recall_up': 75.0, 'precision_down': 95.23809523809523, 'recall_down': 100.0} <----> {'binary_0_1_cutoff_ret_rate_percentage': 0.1, 'learning_rate': 0.05, 'num_epochs': 700, 'batch_size': 70, 'use_bidirectional': False, 'lag': 6, 'input_size': 12, 'hidden_size': 35, 'num_layers': 4, 'use_monthly_dfs_only': True, 'use_binary_0_1_retRate': False, 'use_custom_loss_function_BCE_THRESH': True, 'use_custom_loss_function_BCE_THRESH_AND_SEVERITY': False, 'use_LOW_weights_for_BCE_custom_loss': True, 'pred_threshold_sigmoid01_up': None, 'use_binary_neg1_1': False, 'use_ret_rate': False, 'use_print_acc': False, 'use_dropout': False, 'use_class_weighting': True, 'is_deterministic': True, 'seed_num': 42, 'use_existing_lagged_data': True, 'use_dynamic_weights': False, 'use_binary_0_1_retRate_custom_neg': False, 'use_binary_0_1_retRate_custom_pos': True, 'POS_weight_multiplier': 1.3, 'use_rolling_fixed_train_size': False, 'use_existing_

90

In [ ]:
import random 

### create list of combos for the dist discovery 


all_combos_seeded_dist_disc = []
num_seeds_per_param = 60

for param in combos_1:
    seeds = random.sample(range(int(1e5)) , num_seeds_per_param)
    param = copy.deepcopy(param)
    for ss in seeds:
        param_copy = copy.deepcopy(param)
        param_copy["seed_num"] = ss
        all_combos_seeded_dist_disc.append(param_copy)

len(all_combos_seeded_dist_disc)

with open("all_combos_seeded_dist_disc_NEW.json" , "wb") as f:
    pickle.dump(all_combos_seeded_dist_disc, f)

In [ ]:

### check the number of unique models 

ignore = ['seed_num']
unique = []

for true in all_combos_seeded_dist_disc: ### make sure the combo idx lines up, parallel_backend runs could vary since its the idx when func is called in parallel 

    tt = str({k: v for k, v in true.items() if k not in ignore}).replace(' ', '').replace("'", '').replace('  ', '').replace(',', '')

    unique.append(tt)

len(set(unique))

90

In [ ]:


# --- Run in parallel ---
with parallel_backend("loky", n_jobs=-1):
    results= Parallel()(
        delayed(run_combo_V_4)(i, combo, 0, use_print_acc_vs_pred=False , pred_threshold_sigmoid01_up_bool = False , store_model_weights=False)
        for i, combo in enumerate(all_combos_seeded_dist_disc)
    )


--- Running Combo 7 ---Parameters: {'binary_0_1_cutoff_ret_rate_percentage': 0.1, 'learning_rate': 0.05, 'num_epochs': 700, 'batch_size': 70, 'use_bidirectional': False, 'lag': 6, 'input_size': 12, 'hidden_size': 35, 'num_layers': 4, 'use_monthly_dfs_only': True, 'use_binary_0_1_retRate': False, 'use_custom_loss_function_BCE_THRESH': True, 'use_custom_loss_function_BCE_THRESH_AND_SEVERITY': False, 'use_LOW_weights_for_BCE_custom_loss': True, 'pred_threshold_sigmoid01_up': None, 'use_binary_neg1_1': False, 'use_ret_rate': False, 'use_print_acc': False, 'use_dropout': False, 'use_class_weighting': True, 'is_deterministic': True, 'seed_num': 29256, 'use_existing_lagged_data': True, 'use_dynamic_weights': False, 'use_binary_0_1_retRate_custom_neg': False, 'use_binary_0_1_retRate_custom_pos': True, 'POS_weight_multiplier': 1.3, 'use_rolling_fixed_train_size': False, 'use_existing_initial_weights': False, 'state_dict': None, 'use_USO_wticoncat_predictor_WEEKLY_END_MO': False, 'use_UCO_wticon

KeyboardInterrupt: 

In [ ]:

class np_encoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, pd.Timestamp):
            return obj.isoformat()
        elif isinstance(obj, torch.Tensor):
            return obj.tolist()
        elif isinstance(obj, pd.DatetimeIndex):
            return obj.tolist()
        else:
            return super(np_encoder, self).default(obj)



with open("Vset_NOTsorted_90_models_GS_21_01_to_22_12_DIST_Discovery_10percentPOS_HOD.json" , "w") as f:
    json.dump(results , f , indent=2)

In [ ]:
# ####                DIST DISC T set Random Seeds , SAME MODEL


# ### Dist Disc data
# with open('/Users/cs/Desktop/DATA_LSTM_ETF_V2/top_70_DIST_DISC_GS_20_01_to_21_12_chunk_1_22_10percentPOS_Vset_HOD.json') as f_json:
#     results_dist_disc = json.load(f_json)   


# ## PREDAPRE DATA FRO THE T SET FOR DIST DISCOVERY, SAME PARAMS ARE EXTRACTED , DIFF SEEDS !!!! 


# combos_val_dist_disc_list = [] 

# for entry in results_dist_disc:
#     combo_index = entry["combo_index"]  
#     for i in entry["per_seed_all_results"]:
#         param = i["result_entry"]["parameters"]
#         combos_val_dist_disc_list.append(param)
#         break


# combos_val_dist_disc_list_TEST = copy.deepcopy(combos_val_dist_disc_list)

# for param in combos_val_dist_disc_list_TEST:
#      param['val_start_month'] = '2022-01'
#      param['val_end_month'] = '2022-12'


# import pickle 

# # with open('combos_val_dist_disc_list_TEST.pkl', 'wb') as f:
# #     pickle.dump(combos_val_dist_disc_list_TEST, f)


# with open('combos_val_dist_disc_list_TEST.pkl', 'rb') as f:
#     combos_val_dist_disc_list_TEST = pickle.load(f)



# ### GC RUN


# #### ----------------------                 RUN COMBOS
# with parallel_backend("loky", n_jobs=5):
#     results = Parallel()(
#         delayed(distribution_discovery)(c, combo_index = i , number_of_seeds = 70)
        
#         for i, c in enumerate(combos_val_dist_disc_list_TEST)
#     )



# class NpEncoder(json.JSONEncoder):
#     def default(self, obj):
#         if isinstance(obj, numpy.integer):
#             return int(obj)
#         elif isinstance(obj, numpy.ndarray):
#             return obj.tolist()
#         elif isinstance(obj, pd.DatetimeIndex):
#             return obj.tolist()
#         elif isinstance(obj, pd.Timestamp):
#             return obj.isoformat()
#         else:
#             return super(NpEncoder, self).default(obj)


# file = f"Tset_70_models_GS_22_01_to_22_12_DIST_Discovery_10percentPOS_Vset_HOD.json"

# with open(file, "w") as f_json:
#     json.dump(results, f_json, cls=NpEncoder, indent=2)

In [ ]:

with open("/Users/cs/Desktop/DATA_LSTM_ETF_V2/Vset_NOTsorted_90_models_GS_21_01_to_22_12_DIST_Discovery_10percentPOS_HOD.json" , "r") as f:
    results_dist_disc_Vset_NOTsorted_same_seeds = json.load(f)

results_dist_disc_Vset_NOTsorted_same_seeds_copy = copy.deepcopy(results_dist_disc_Vset_NOTsorted_same_seeds)
results_dist_disc = []
ignore_find_combo = ['seed_num']

combo_idxs = np.arange(len(results_dist_disc_Vset_NOTsorted_same_seeds_copy) + 1).tolist()
combo_idxs_counter = 0

for idx ,entry in enumerate(results_dist_disc_Vset_NOTsorted_same_seeds_copy):
    organized_results_entry = {} ## entry for results_dist_disc
    params_done = [e['combo'] for e in results_dist_disc] ## for each entry, there is a combo , these are unique
    params_done_keys = [str({k: v for k, v in c.items() if k not in ignore_find_combo}).replace(' ', '').replace("'", '').replace('  ', '').replace(',', '') for c in params_done]
    current_param_keys = str({k: v for k, v in entry['parameters'].items() if k not in ignore_find_combo}).replace(' ', '').replace("'", '').replace('  ', '').replace(',', '')
    if current_param_keys not in params_done_keys:
        combo_idxs_counter += 1
        organized_results_entry["combo_index"] = combo_idxs[combo_idxs_counter]
        organized_results_entry["combo"] = entry["parameters"]
        organized_results_entry["per_seed_all_results"] = []
        for params_for_combo in results_dist_disc_Vset_NOTsorted_same_seeds_copy:
            seed_results_entry = {}
            if str({k: v for k, v in params_for_combo['parameters'].items() if k not in ignore_find_combo}).replace(' ', '').replace("'", '').replace('  ', '').replace(',', '') == current_param_keys:
                seed_results_entry["seed"] = params_for_combo["parameters"]["seed_num"]
                seed_results_entry["result_entry"] = params_for_combo
                organized_results_entry["per_seed_all_results"].append(seed_results_entry)
        results_dist_disc.append(organized_results_entry)




In [ ]:

#####                 DIST DISCOVERY  DATA T SET WITH THE SAME SEED SBUT FOR T SET, PREV IT WAS DONE WITH DIFF SEEDS 

### extract all params , chantge to T set  , run GC c4d , combina again later 

all_preds_dist_disc_val = []

results_dist_disc_copy = copy.deepcopy(results_dist_disc)
for i in results_dist_disc_copy:
    for res in i["per_seed_all_results"]:
        # print(res["result_entry"]["parameters"])
        all_preds_dist_disc_val.append(res["result_entry"]["parameters"])

len(all_preds_dist_disc_val)  

all_params_dist_disc_TESTset_SAME_SEEDSasVal = copy.deepcopy(all_preds_dist_disc_val)
for param in all_params_dist_disc_TESTset_SAME_SEEDSasVal:
     param['val_start_month'] = '2023-01'
     param['val_end_month'] = '2023-12'

with open('all_params_dist_disc_TESTset_SAME_SEEDSasVal.pkl', 'wb') as f:
    pickle.dump(all_params_dist_disc_TESTset_SAME_SEEDSasVal, f)


######### GC RUN 


In [ ]:

with open('all_params_dist_disc_TESTset_SAME_SEEDSasVal.pkl', 'rb') as f:
    all_params_dist_disc_TESTset_SAME_SEEDSasVal = pickle.load(f)


[{'binary_0_1_cutoff_ret_rate_percentage': 0.1,
  'learning_rate': 0.05,
  'num_epochs': 700,
  'batch_size': 70,
  'use_bidirectional': False,
  'lag': 6,
  'input_size': 12,
  'hidden_size': 35,
  'num_layers': 4,
  'use_monthly_dfs_only': True,
  'use_binary_0_1_retRate': False,
  'use_custom_loss_function_BCE_THRESH': True,
  'use_custom_loss_function_BCE_THRESH_AND_SEVERITY': False,
  'use_LOW_weights_for_BCE_custom_loss': True,
  'pred_threshold_sigmoid01_up': None,
  'use_binary_neg1_1': False,
  'use_ret_rate': False,
  'use_print_acc': False,
  'use_dropout': False,
  'use_class_weighting': True,
  'is_deterministic': True,
  'seed_num': 53558,
  'use_existing_lagged_data': True,
  'use_dynamic_weights': False,
  'use_binary_0_1_retRate_custom_neg': False,
  'use_binary_0_1_retRate_custom_pos': True,
  'POS_weight_multiplier': 1.3,
  'use_rolling_fixed_train_size': False,
  'use_existing_initial_weights': False,
  'state_dict': None,
  'use_USO_wticoncat_predictor_WEEKLY_END_M

In [ ]:


#### ----------------------                 RUN COMBOS

with parallel_backend("loky", n_jobs=-1):
    RES___all_params_dist_disc_TESTset_SAME_SEEDSasVal = Parallel()(
        delayed(run_combo_V_4)(i, combo, 0, use_print_acc_vs_pred=False , pred_threshold_sigmoid01_up_bool = False  , store_model_weights = False )
        for i, combo in enumerate(all_params_dist_disc_TESTset_SAME_SEEDSasVal)
    )


class np_encoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, pd.Timestamp):
            return obj.isoformat()
        elif isinstance(obj, torch.Tensor):
            return obj.tolist()
        elif isinstance(obj, pd.DatetimeIndex):
            return obj.tolist()
        else:
            return super(np_encoder, self).default(obj)


with open("Tset_NOTsorted_90_models_GS_21_01_to_22_12_DIST_Discovery_10percentPOS_HOD.json", "w") as f_json:
    json.dump(RES___all_params_dist_disc_TESTset_SAME_SEEDSasVal, f_json , indent=2, cls = np_encoder)



---
# UCO 21_01 - 22_12
---

In [26]:
####                                       First Dist Disc   - Ran top models


with open("/Users/cs/Desktop/DATA_LSTM_ETF_V2/GS_21_01_to_22_12_chunk_1_32_10percentPOS_Vset_UCO.json" , "rb") as f:
    results = json.load(f)


scored_results = []
for entry in results:
    avg = entry['cv_sets'].get('overall_metrics')
    acc = avg.get('accuracy') or 0.0
    prec_up = avg.get('precision_up') or 0.0
    recall_up = avg.get('recall_up') or 0.0

    total_score = acc  # or add prec_up, etc.
    
    if recall_up > 1 and prec_up > 40:

        scored_results.append((total_score, entry))

# Step 2: Sort by score descending
scored_results.sort(reverse=True, key=lambda x: x[0])


top = scored_results[:90]

for i, j in top:
    print(j["combo_number"], '<--->' , j["cv_sets"]["overall_metrics"] , '<---->' , j["parameters"] )

# Step 4: Extract the parameter combos
combos_1 = [j["parameters"] for i, j in top]


len(combos_1)


# combos_test_param = []

# for i in combos:
#     i_copy = copy.deepcopy(i)
#     i_copy["val_set_fraction"] = 0.05
#     i_copy["num_folds"] = 4
#     i_copy["end_value_train_set_fraction"] = 0.95
#     combos_test_param.append(i_copy)


# #### ----------------------                 RUN COMBOS
# with parallel_backend("loky", n_jobs=3):
#     results = Parallel()(
#         delayed(distribution_discovery)(c, combo_index = i , number_of_seeds = 60)
        
#         for i, c in enumerate(combos)
#     )



1245 <---> {'accuracy': 79.16666666666666, 'precision_up': 87.5, 'recall_up': 63.63636363636363, 'precision_down': 75.0, 'recall_down': 92.3076923076923} <----> {'binary_0_1_cutoff_ret_rate_percentage': 0.1, 'learning_rate': 0.0005, 'num_epochs': 700, 'batch_size': 50, 'use_bidirectional': False, 'lag': 2, 'input_size': 12, 'hidden_size': 35, 'num_layers': 6, 'use_monthly_dfs_only': True, 'use_binary_0_1_retRate': False, 'use_custom_loss_function_BCE_THRESH': False, 'use_custom_loss_function_BCE_THRESH_AND_SEVERITY': True, 'use_LOW_weights_for_BCE_custom_loss': True, 'pred_threshold_sigmoid01_up': None, 'use_binary_neg1_1': False, 'use_ret_rate': False, 'use_print_acc': False, 'use_dropout': False, 'use_class_weighting': True, 'is_deterministic': True, 'seed_num': 42, 'use_existing_lagged_data': True, 'use_dynamic_weights': False, 'use_binary_0_1_retRate_custom_neg': False, 'use_binary_0_1_retRate_custom_pos': True, 'POS_weight_multiplier': 1.3, 'use_rolling_fixed_train_size': False, '

90

In [39]:
import random 

### create list of combos for the dist discovery 


all_combos_seeded_dist_disc = []
num_seeds_per_param = 40

for param in combos_1:
    seeds = random.sample(range(int(1e5)) , num_seeds_per_param)
    param = copy.deepcopy(param)
    for ss in seeds:
        param_copy = copy.deepcopy(param)
        param_copy["seed_num"] = ss
        all_combos_seeded_dist_disc.append(param_copy)

len(all_combos_seeded_dist_disc)

with open("all_combos_seeded_dist_disc_NEW.pkl" , "wb") as f:
    pickle.dump(all_combos_seeded_dist_disc, f)

len(all_combos_seeded_dist_disc)

3600

In [ ]:

### check the number of unique models 

ignore = ['seed_num']
unique = []

for true in all_combos_seeded_dist_disc: ### make sure the combo idx lines up, parallel_backend runs could vary since its the idx when func is called in parallel 

    tt = str({k: v for k, v in true.items() if k not in ignore}).replace(' ', '').replace("'", '').replace('  ', '').replace(',', '')

    unique.append(tt)


len([i for i in all_combos_seeded_dist_disc if i['num_epochs'] == 700]) 


28.0

In [27]:


with open('all_combos_seeded_dist_disc_NEW.pkl' , 'rb') as f:
    all_combos_seeded_dist_disc = pickle.load(f)


# --- Run in parallel ---
with parallel_backend("loky", n_jobs=-1):
    results= Parallel()(
        delayed(run_combo_V_4)(i, combo, 0, use_print_acc_vs_pred=False , pred_threshold_sigmoid01_up_bool = False , store_model_weights=False)
        for i, combo in enumerate(all_combos_seeded_dist_disc)
    )

#HERE2

FileNotFoundError: [Errno 2] No such file or directory: 'all_combos_seeded_dist_disc_NEW.pkl'

In [24]:

class np_encoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, pd.Timestamp):
            return obj.isoformat()
        elif isinstance(obj, torch.Tensor):
            return obj.tolist()
        elif isinstance(obj, pd.DatetimeIndex):
            return obj.tolist()
        else:
            return super(np_encoder, self).default(obj)



with open("Vset_NOTsorted_90_models_GS_21_01_to_22_12_DIST_Discovery_10percentPOS_UCO.json" , "w") as f:
    json.dump(results , f , indent=2)



In [29]:

all_params_dist_disc_TESTset_SAME_SEEDSasVal = copy.deepcopy(all_combos_seeded_dist_disc)
for param in all_params_dist_disc_TESTset_SAME_SEEDSasVal:
     param['val_start_month'] = '2023-01'
     param['val_end_month'] = '2023-12'

with open('all_params_dist_disc_TESTset_SAME_SEEDSasVal.pkl', 'wb') as f:
    pickle.dump(all_params_dist_disc_TESTset_SAME_SEEDSasVal, f)

In [ ]:
with open("all_params_dist_disc_TESTset_SAME_SEEDSasVal.pkl", "rb") as f:
    all_params_dist_disc_TESTset_SAME_SEEDSasVal = pickle.load(f)

with parallel_backend("loky", n_jobs=-1):
    res = Parallel()(
        delayed(run_combo_V_4)(i, combo, 0, use_print_acc_vs_pred=False , pred_threshold_sigmoid01_up_bool = False  , store_model_weights = False )
        for i, combo in enumerate(all_params_dist_disc_TESTset_SAME_SEEDSasVal)
    )



#HERE2

In [ ]:

class np_encoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, pd.Timestamp):
            return obj.isoformat()
        elif isinstance(obj, torch.Tensor):
            return obj.tolist()
        elif isinstance(obj, pd.DatetimeIndex):
            return obj.tolist()
        else:
            return super(np_encoder, self).default(obj)


with open("Tset_NOTsorted_90_models_GS_21_01_to_22_12_DIST_Discovery_10percentPOS_UCO.json", "w") as f_json:
    json.dump(res, f_json , indent=2, cls = np_encoder)


---
# HOD 22_01 - 23_12
---

In [ ]:
import json

In [ ]:
####                                       First Dist Disc   - Ran top models


with open("/Users/cs/Desktop/DATA_LSTM_ETF_V2/GS_22_01_to_23_12_chunk_1_32_10percentPOS_Vset_HOD.json" , "rb") as f:
    results = json.load(f)


scored_results = []
for entry in results:
    avg = entry['cv_sets'].get('overall_metrics')
    acc = avg.get('accuracy') or 0.0
    prec_up = avg.get('precision_up') or 0.0
    recall_up = avg.get('recall_up') or 0.0

    total_score = acc  # or add prec_up, etc.
    
    if recall_up > 1 and prec_up > 40:

        scored_results.append((total_score, entry))

# Step 2: Sort by score descending
scored_results.sort(reverse=True, key=lambda x: x[0])


top = scored_results[:90]

for i, j in top:
    print(j["combo_number"], '<--->' , j["cv_sets"]["overall_metrics"] , '<---->' , j["parameters"] )

# Step 4: Extract the parameter combos
combos_2 = [j["parameters"] for i, j in top]


len(combos_2)
# combos_test_param = []

# for i in combos:
#     i_copy = copy.deepcopy(i)
#     i_copy["val_set_fraction"] = 0.05
#     i_copy["num_folds"] = 4
#     i_copy["end_value_train_set_fraction"] = 0.95
#     combos_test_param.append(i_copy)


# #### ----------------------                 RUN COMBOS
# with parallel_backend("loky", n_jobs=3):
#     results = Parallel()(
#         delayed(distribution_discovery)(c, combo_index = i , number_of_seeds = 60)
        
#         for i, c in enumerate(combos)
#     )






109 <---> {'accuracy': 91.66666666666666, 'precision_up': 75.0, 'recall_up': 75.0, 'precision_down': 95.0, 'recall_down': 95.0} <----> {'binary_0_1_cutoff_ret_rate_percentage': 0.1, 'learning_rate': 0.005, 'num_epochs': 70, 'batch_size': 30, 'use_bidirectional': False, 'lag': 8, 'input_size': 12, 'hidden_size': 35, 'num_layers': 2, 'use_monthly_dfs_only': True, 'use_binary_0_1_retRate': False, 'use_custom_loss_function_BCE_THRESH': True, 'use_custom_loss_function_BCE_THRESH_AND_SEVERITY': False, 'use_LOW_weights_for_BCE_custom_loss': True, 'pred_threshold_sigmoid01_up': None, 'use_binary_neg1_1': False, 'use_ret_rate': False, 'use_print_acc': False, 'use_dropout': False, 'use_class_weighting': False, 'is_deterministic': True, 'seed_num': 42, 'use_existing_lagged_data': True, 'use_dynamic_weights': False, 'use_binary_0_1_retRate_custom_neg': False, 'use_binary_0_1_retRate_custom_pos': True, 'POS_weight_multiplier': 1, 'use_rolling_fixed_train_size': False, 'use_existing_initial_weights'

90

In [ ]:
import random 

### create list of combos for the dist discovery 

all_combos_seeded_dist_disc = []
num_seeds_per_param = 60

for param in combos_2: 
    seeds = random.sample(range(int(1e5)) , num_seeds_per_param)
    param = copy.deepcopy(param)
    for ss in seeds:
        param_copy = copy.deepcopy(param)
        param_copy["seed_num"] = ss
        all_combos_seeded_dist_disc.append(param_copy)

len(all_combos_seeded_dist_disc)

with open("VSETall_combos_seeded_dist_disc_NEW_22_01_to_23_12_NEW.pkl" , "wb") as f:
    pickle.dump(all_combos_seeded_dist_disc, f)

In [ ]:

### check the number of unique models 

ignore = ['seed_num']
unique = []

for true in all_combos_seeded_dist_disc: ### make sure the combo idx lines up, parallel_backend runs could vary since its the idx when func is called in parallel 

    tt = str({k: v for k, v in true.items() if k not in ignore}).replace(' ', '').replace("'", '').replace('  ', '').replace(',', '')

    unique.append(tt)

len(set(unique))

90

In [ ]:
with open("VSETall_combos_seeded_dist_disc_NEW_22_01_to_23_12_NEW.pkl" , "rb") as f:
    all_combos_seeded_dist_disc = pickle.load(f)

# --- Run in parallel ---
with parallel_backend("loky", n_jobs=-1):
    results= Parallel()(
        delayed(run_combo_V_4)(i, combo, 0, use_print_acc_vs_pred=False , pred_threshold_sigmoid01_up_bool = False , store_model_weights=False)
        for i, combo in enumerate(all_combos_seeded_dist_disc)
    )




5400

In [ ]:

class np_encoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, pd.Timestamp):
            return obj.isoformat()
        elif isinstance(obj, torch.Tensor):
            return obj.tolist()
        elif isinstance(obj, pd.DatetimeIndex):
            return obj.tolist()
        else:
            return super(np_encoder, self).default(obj)



with open("Vset_NOTsorted_90_models_GS_22_01_to_23_12_DIST_Discovery_10percentPOS_HOD.json" , "w") as f:
    json.dump(results , f , indent=2 , cls = np_encoder)

In [ ]:

#####                 DIST DISCOVERY  DATA T SET WITH THE SAME SEED SBUT FOR T SET, PREV IT WAS DONE WITH DIFF SEEDS 

### extract all params , chantge to T set  , run GC c4d , combina again later 

# all_preds_dist_disc_val = []

# results_dist_disc_copy = copy.deepcopy(results_dist_disc)
# for i in results_dist_disc_copy:
#     for res in i["per_seed_all_results"]:
#         # print(res["result_entry"]["parameters"])
#         all_preds_dist_disc_val.append(res["result_entry"]["parameters"])

# len(all_preds_dist_disc_val)  


with open("VSETall_combos_seeded_dist_disc_NEW_22_01_to_23_12_NEW.pkl" , "rb") as f:
    all_combos_seeded_dist_disc = pickle.load(f)


all_params_dist_disc_TESTset_SAME_SEEDSasVal = copy.deepcopy(all_combos_seeded_dist_disc)
for param in all_params_dist_disc_TESTset_SAME_SEEDSasVal:
     param['val_start_month'] = '2024-01'
     param['val_end_month'] = '2024-12'

with open('TSETall_combos_seeded_dist_disc_NEW_22_01_to_23_12_NEW.pkl', 'wb') as f:
    pickle.dump(all_params_dist_disc_TESTset_SAME_SEEDSasVal, f)


######### GC RUN 


In [ ]:
with open('TSETall_combos_seeded_dist_disc_NEW_22_01_to_23_12_NEW.pkl', 'rb') as f:
    all_combos_seeded_dist_disc = pickle.load(f)


# --- Run in parallel ---
with parallel_backend("loky", n_jobs=-1):
    results= Parallel()(
        delayed(run_combo_V_4)(i, combo, 0, use_print_acc_vs_pred=False , pred_threshold_sigmoid01_up_bool = False , store_model_weights=False)
        for i, combo in enumerate(all_combos_seeded_dist_disc)
    )




In [ ]:
class np_encoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, pd.Timestamp):
            return obj.isoformat()
        elif isinstance(obj, torch.Tensor):
            return obj.tolist()
        elif isinstance(obj, pd.DatetimeIndex):
            return obj.tolist()
        else:
            return super(np_encoder, self).default(obj)



with open("Tset_NOTsorted_90_models_GS_22_01_to_23_12_DIST_Discovery_10percentPOS_HOD.json" , "w") as f:
    json.dump(results , f , indent=2 , cls = np_encoder)

In [ ]:
params_UCO[0]

{'binary_0_1_cutoff_ret_rate_percentage': 0.1,
 'learning_rate': 0.0005,
 'num_epochs': 700,
 'batch_size': 50,
 'use_bidirectional': False,
 'lag': 8,
 'input_size': 12,
 'hidden_size': 12,
 'num_layers': 2,
 'use_monthly_dfs_only': True,
 'use_binary_0_1_retRate': False,
 'use_custom_loss_function_BCE_THRESH': False,
 'use_custom_loss_function_BCE_THRESH_AND_SEVERITY': False,
 'use_LOW_weights_for_BCE_custom_loss': True,
 'pred_threshold_sigmoid01_up': None,
 'use_binary_neg1_1': False,
 'use_ret_rate': False,
 'use_print_acc': False,
 'use_dropout': False,
 'use_class_weighting': False,
 'is_deterministic': True,
 'seed_num': 42,
 'use_existing_lagged_data': True,
 'use_dynamic_weights': False,
 'use_binary_0_1_retRate_custom_neg': False,
 'use_binary_0_1_retRate_custom_pos': True,
 'POS_weight_multiplier': 0.7,
 'use_rolling_fixed_train_size': False,
 'use_existing_initial_weights': False,
 'state_dict': None,
 'use_USO_wticoncat_predictor_WEEKLY_END_MO': False,
 'use_UCO_wticoncat

In [ ]:
# #####                 DIST DISCOVERY  DATA T SET WITH THE SAME SEED SBUT FOR T SET, PREV IT WAS DONE WITH DIFF SEEDS 

# ### extract all params , chantge to T set  , run GC c4d , combina again later 

# all_preds_dist_disc_val = []

# for i in results_dist_disc:
#     for res in i["per_seed_all_results"]:
#         # print(res["result_entry"]["parameters"])
#         all_preds_dist_disc_val.append(res["result_entry"]["parameters"])

# len(all_preds_dist_disc_val)  

# all_params_dist_disc_TESTset_SAME_SEEDSasVal = copy.deepcopy(all_preds_dist_disc_val)
# for param in all_params_dist_disc_TESTset_SAME_SEEDSasVal:
#      param['val_start_month'] = '2023-01'
#      param['val_end_month'] = '2023-12'

# with open('all_params_dist_disc_TESTset_SAME_SEEDSasVal_21_01__22_12.pkl', 'wb') as f:
#     pickle.dump(all_params_dist_disc_TESTset_SAME_SEEDSasVal, f)


# ######### GC RUN 


# with open('all_params_dist_disc_TESTset_SAME_SEEDSasVal_21_01__22_12.pkl', 'rb') as f:
#     all_params_dist_disc_TESTset_SAME_SEEDSasVal = pickle.load(f)


# # #### ----------------------                 RUN COMBOS

# with parallel_backend("loky", n_jobs=-1):
#     RES___all_params_dist_disc_TESTset_SAME_SEEDSasVal = Parallel()(
#         delayed(run_combo_V_4)(i, combo, 0, use_print_acc_vs_pred=False , pred_threshold_sigmoid01_up_bool = False  , store_model_weights = False )
#         for i, combo in enumerate(all_params_dist_disc_TESTset_SAME_SEEDSasVal)
#     )

    

# class np_encoder(json.JSONEncoder):
#     def default(self, obj):
#         if isinstance(obj, np.integer):
#             return int(obj)
#         elif isinstance(obj, np.floating):
#             return float(obj)
#         elif isinstance(obj, np.ndarray):
#             return obj.tolist()
#         else:
#             return super(np_encoder, self).default(obj)

# with open("Tset_NOTsorted_70_models_GS_21_01_to_22_12_DIST_Discovery_10percentPOS_Vset_HOD.json", "w") as f_json:
#     json.dump(RES___all_params_dist_disc_TESTset_SAME_SEEDSasVal, f_json , indent=2, cls = np_encoder)



In [ ]:

# with open('all_params_dist_disc_TESTset_SAME_SEEDSasVal_21_01__22_12.pkl', 'rb') as f:
#     all_params_dist_disc_TESTset_SAME_SEEDSasVal = pickle.load(f)


# # #### ----------------------                 RUN COMBOS

# with parallel_backend("loky", n_jobs=-1):
#     RES___all_params_dist_disc_TESTset_SAME_SEEDSasVal = Parallel()(
#         delayed(run_combo_V_4)(i, combo, 0, use_print_acc_vs_pred=False , pred_threshold_sigmoid01_up_bool = False  , store_model_weights = False )
#         for i, combo in enumerate(all_params_dist_disc_TESTset_SAME_SEEDSasVal)
#     )



# class np_encoder(json.JSONEncoder):
#     def default(self, obj):
#         if isinstance(obj, np.integer):
#             return int(obj)
#         elif isinstance(obj, np.floating):
#             return float(obj)
#         elif isinstance(obj, np.ndarray):
#             return obj.tolist()
#         elif isinstance(obj, pd.Timestamp):
#             return obj.isoformat()
#         elif isinstance(obj, torch.Tensor):
#             return obj.tolist()
#         elif isinstance(obj, pd.DatetimeIndex):
#             return obj.tolist()
#         else:
#             return super(np_encoder, self).default(obj)


# with open("Tset_NOTsorted_70_models_GS_21_01_to_22_12_DIST_Discovery_10percentPOS_Vset_HOD.json", "w") as f_json:
#     json.dump(RES___all_params_dist_disc_TESTset_SAME_SEEDSasVal, f_json , indent=2, cls = np_encoder)



---
# UCO 22_01 - 23_12
---

In [ ]:
# import copy


# with open("/Users/cs/Desktop/DATA_LSTM_ETF_V2/GS_22_01_to_23_12_chunk_1_32_10percentPOS_Vset_HOD.json" , "rb") as f:
#     results = json.load(f)

# all_params = [i["parameters"] for i in results]

# params_UCO = []
# for i in copy.deepcopy(all_params):
#     i["use_UCO_wticoncat_predictor_WEEKLY_END_MO"] = True
#     i["use_HOD_wticoncat_predictor_WEEKLY_END_MO"] = False
#     params_UCO.append(i)

# len(all_params)

# with open('params_UCO_GS_22_01_to_23_12.pkl', 'wb') as f:
#     pickle.dump(params_UCO, f)

In [1]:
# with open('params_UCO_GS_22_01_to_23_12.pkl', 'rb') as f:
#    combos = pickle.load(f)


# # --- Run in parallel ---
# with parallel_backend("loky", n_jobs=-1):
#     results= Parallel()(
#         delayed(run_combo_V_4)(i, combo, 0, use_print_acc_vs_pred=False , pred_threshold_sigmoid01_up_bool = False , store_model_weights=False)
#         for i, combo in enumerate(combos)
#     )



In [ ]:

# class np_encoder(json.JSONEncoder):
#     def default(self, obj):
#         if isinstance(obj, np.integer):
#             return int(obj)
#         elif isinstance(obj, np.floating):
#             return float(obj)
#         elif isinstance(obj, np.ndarray):
#             return obj.tolist()
#         elif isinstance(obj, pd.Timestamp):
#             return obj.isoformat()
#         elif isinstance(obj, torch.Tensor):
#             return obj.tolist()
#         elif isinstance(obj, pd.DatetimeIndex):
#             return obj.tolist()
#         else:
#             return super(np_encoder, self).default(obj)



# with open("Vset_NOTsorted_90_models_GS_22_01_to_23_12_DIST_Discovery_10percentPOS_UCO.json" , "w") as f:
#     json.dump(results , f , indent=2)

In [8]:
####                                       First Dist Disc   - Ran top models
import json

with open("/Users/cs/Desktop/DATA_LSTM_ETF_V2/GS_22_01_to_23_12_chunk_1_32_10percentPOS_Vset_UCO.json" , "rb") as f:
    results = json.load(f)


scored_results = []
for entry in results:
    avg = entry['cv_sets'].get('overall_metrics')
    acc = avg.get('accuracy') or 0.0
    prec_up = avg.get('precision_up') or 0.0
    recall_up = avg.get('recall_up') or 0.0

    total_score = acc  # or add prec_up, etc.
    
    if recall_up > 1 and prec_up > 40:

        scored_results.append((total_score, entry))

# Step 2: Sort by score descending
scored_results.sort(reverse=True, key=lambda x: x[0])


top = scored_results[:90]

for i, j in top:
    print(j["combo_number"], '<--->' , j["cv_sets"]["overall_metrics"] , '<---->' , j["parameters"] )

# Step 4: Extract the parameter combos
combos_3 = [j["parameters"] for i, j in top]


len(combos_3)


# combos_test_param = []

# for i in combos:
#     i_copy = copy.deepcopy(i)
#     i_copy["val_set_fraction"] = 0.05
#     i_copy["num_folds"] = 4
#     i_copy["end_value_train_set_fraction"] = 0.95
#     combos_test_param.append(i_copy)


# #### ----------------------                 RUN COMBOS
# with parallel_backend("loky", n_jobs=3):
#     results = Parallel()(
#         delayed(distribution_discovery)(c, combo_index = i , number_of_seeds = 60)
        
#         for i, c in enumerate(combos)
#     )



2319 <---> {'accuracy': 87.5, 'precision_up': 100.0, 'recall_up': 57.14285714285714, 'precision_down': 85.0, 'recall_down': 100.0} <----> {'binary_0_1_cutoff_ret_rate_percentage': 0.1, 'learning_rate': 0.005, 'num_epochs': 700, 'batch_size': 50, 'use_bidirectional': False, 'lag': 2, 'input_size': 12, 'hidden_size': 12, 'num_layers': 6, 'use_monthly_dfs_only': True, 'use_binary_0_1_retRate': False, 'use_custom_loss_function_BCE_THRESH': False, 'use_custom_loss_function_BCE_THRESH_AND_SEVERITY': False, 'use_LOW_weights_for_BCE_custom_loss': True, 'pred_threshold_sigmoid01_up': None, 'use_binary_neg1_1': False, 'use_ret_rate': False, 'use_print_acc': False, 'use_dropout': False, 'use_class_weighting': False, 'is_deterministic': True, 'seed_num': 42, 'use_existing_lagged_data': True, 'use_dynamic_weights': False, 'use_binary_0_1_retRate_custom_neg': False, 'use_binary_0_1_retRate_custom_pos': True, 'POS_weight_multiplier': 1, 'use_rolling_fixed_train_size': False, 'use_existing_initial_wei

90

In [9]:
import random 
import copy
import pickle

### create list of combos for the dist discovery 

all_combos_seeded_dist_disc = []
num_seeds_per_param = 60

for param in combos_3: 
    seeds = random.sample(range(int(1e5)) , num_seeds_per_param)
    param = copy.deepcopy(param)
    for ss in seeds:
        param_copy = copy.deepcopy(param)
        param_copy["seed_num"] = ss
        all_combos_seeded_dist_disc.append(param_copy)

len(all_combos_seeded_dist_disc)

with open("UCO__VSETall_combos_seeded_dist_disc.pkl" , "wb") as f:
    pickle.dump(all_combos_seeded_dist_disc, f)

In [10]:

### check the number of unique models 

ignore = ['seed_num']
unique = []

for true in all_combos_seeded_dist_disc: ### make sure the combo idx lines up, parallel_backend runs could vary since its the idx when func is called in parallel 

    tt = str({k: v for k, v in true.items() if k not in ignore}).replace(' ', '').replace("'", '').replace('  ', '').replace(',', '')

    unique.append(tt)

len(set(unique))

90

In [19]:
with open("UCO__VSETall_combos_seeded_dist_disc.pkl" , "rb") as f:
    all_combos_seeded_dist_disc = pickle.load(f)

with parallel_backend("loky", n_jobs=-1):
    res_1 = Parallel()(
        delayed(run_combo_V_4)(i, combo, 0, use_print_acc_vs_pred=False , pred_threshold_sigmoid01_up_bool = False  , store_model_weights = False )
        for i, combo in enumerate(all_combos_seeded_dist_disc)
    )




KeyboardInterrupt: 

In [ ]:

class np_encoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, pd.Timestamp):
            return obj.isoformat()
        elif isinstance(obj, torch.Tensor):
            return obj.tolist()
        elif isinstance(obj, pd.DatetimeIndex):
            return obj.tolist()
        else:
            return super(np_encoder, self).default(obj)



with open("Vset_NOTsorted_90_models_GS_22_01_to_23_12_DIST_Discovery_10percentPOS_UCO.json" , "w") as f:
    json.dump(res_1 , f , indent=2 , cls = np_encoder )

In [12]:

with open("UCO__VSETall_combos_seeded_dist_disc.pkl" , "rb") as f:
    all_combos_seeded_dist_disc = pickle.load(f)

all_combos_seeded_dist_disc_TSET = copy.deepcopy(all_combos_seeded_dist_disc)

for param in all_combos_seeded_dist_disc_TSET:
     param['val_start_month'] = '2024-01'
     param['val_end_month'] = '2024-12'

with open('T_UCO_DISTall_params_dist_disc_TESTset_SAME_SEEDSasVal.pkl', 'wb') as f:
    pickle.dump(all_combos_seeded_dist_disc_TSET, f)

In [ ]:
with open("T_UCO_DISTall_params_dist_disc_TESTset_SAME_SEEDSasVal.pkl" , "rb") as f:
    params = pickle.load(f)

#### ----------------------                 RUN COMBOS

with parallel_backend("loky", n_jobs=-1):
    res = Parallel()(
        delayed(run_combo_V_4)(i, combo, 0, use_print_acc_vs_pred=False , pred_threshold_sigmoid01_up_bool = False  , store_model_weights = False )
        for i, combo in enumerate(params)
    )



In [ ]:

class np_encoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, pd.Timestamp):
            return obj.isoformat()
        elif isinstance(obj, torch.Tensor):
            return obj.tolist()
        elif isinstance(obj, pd.DatetimeIndex):
            return obj.tolist()
        else:
            return super(np_encoder, self).default(obj)


with open("Tset_NOTsorted_90_models_GS_22_01_to_23_12_DIST_Discovery_10percentPOS_UCO.json", "w") as f_json:
    json.dump(res, f_json , indent=2, cls = np_encoder)

